In [17]:
class ToDictMixin:
    def to_dict(self):
        return self._traverse_dict(self.__dict__)
    
    def _traverse_dict(self, instance_dict):
        output = {}
        for key, value in instance_dict.items():
            output[key] = self._traverse(key, value)
        return output
    
    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        elif hasattr(value, '__dict__'):
            return self._traverse_dict(value.__dict__)
        else:
            return value
        
class BinaryTree(ToDictMixin):
    def __init__(self, value, left=None, right=None):
        self.value= value
        self.left = left
        self.right = right

In [18]:
tree = BinaryTree(10,
                  left=BinaryTree(7, right=BinaryTree(9)),
                  right=BinaryTree(13, left=BinaryTree(11)))
print(tree.to_dict())

{'value': 10, 'left': {'value': 7, 'left': None, 'right': {'value': 9, 'left': None, 'right': None}}, 'right': {'value': 13, 'left': {'value': 11, 'left': None, 'right': None}, 'right': None}}


The best part about mix-ins is that you can make their generic functionality pluggable so behaviors can be overridden when required.

In [30]:
class BinaryTreeWithParent(BinaryTree):
    def __init__(self, value, left=None,
                 right=None, parent=None):
        super().__init__(value, left=left, right=right)
        self.parent = parent
    #the solution is to override the BinaryTreeWithParent._traverse method
    #to only process values that matter, preventing cycles encountered by
    #the mix-in
    def _traverse(self, key, value):
        print('parent')
        if(isinstance(value, BinaryTreeWithParent) and
              key == 'parent'):
            return value.value  # Prevent cycles
        else:
            return super()._traverse(key, value)

In [31]:
root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent=root)
root.left.right = BinaryTreeWithParent(9, parent=root.left)
for i, value in root.to_dict().items():
    print(f'{repr(i)}: {value},')

parent
parent
parent
parent
parent
parent
parent
parent
parent
parent
parent
parent
'value': 10,
'left': {'value': 7, 'left': None, 'right': {'value': 9, 'left': None, 'right': None, 'parent': 7}, 'parent': 10},
'right': None,
'parent': None,


By defining BinaryTreeWithParent._traverse, I've also enabled any class that has an attribute of BinaryTreeWithParent to automatically work with the ToDictMixin

In [33]:
class NamedSubTree(ToDictMixin):
    def __init__(self, name, tree_with_parent):
        self.name = name
        self.tree_with_parent = tree_with_parent
        
my_tree = NamedSubTree('foobar', root.left.right) 
print(my_tree.to_dict())

parent
parent
parent
parent
{'name': 'foobar', 'tree_with_parent': {'value': 9, 'left': None, 'right': None, 'parent': 7}}


In [34]:
import json

class JsonMixin:
    @classmethod
    def from_json(cls, data):
        kwargs = json.loads(data)
        return cls(**kwargs)
    
    def to_json(self):
        return json.dumps(self.to_dict())

In [35]:
class DatacenterRack(ToDictMixin, JsonMixin):
    def __init__(self, switch=None, machines=None):
        self.switch = Switch(**switch)
        self.machines = [
            Machine(**kwargs) for kwargs in machines]
        

class Switch(ToDictMixin, JsonMixin):
    def __init__(self, ports=None, speed=None):
        self.ports = ports
        self.speed = speed
        

class Machine(ToDictMixin, JsonMixin):
    def __init__(self, cores=None, ram=None, disk=None):
        self.cores = cores
        self.ram = ram
        self.disk = disk

In [38]:
serialized = """{
    "switch": {"ports": 5, "speed": 1e9},
    "machines":[
        {"cores": 8, "ram": 32e9, "disk": 5e12},
        {"cores": 4, "ram": 16e9, "disk": 1e12},
        {"cores": 2, "ram": 4e9, "disk": 500e9}
    ]
}"""

deserialized = DatacenterRack.from_json(serialized)
roundtrip = deserialized.to_json()

print(json.loads(serialized))
print(json.loads(roundtrip))

{'switch': {'ports': 5, 'speed': 1000000000.0}, 'machines': [{'cores': 8, 'ram': 32000000000.0, 'disk': 5000000000000.0}, {'cores': 4, 'ram': 16000000000.0, 'disk': 1000000000000.0}, {'cores': 2, 'ram': 4000000000.0, 'disk': 500000000000.0}]}
{'switch': {'ports': 5, 'speed': 1000000000.0}, 'machines': [{'cores': 8, 'ram': 32000000000.0, 'disk': 5000000000000.0}, {'cores': 4, 'ram': 16000000000.0, 'disk': 1000000000000.0}, {'cores': 2, 'ram': 4000000000.0, 'disk': 500000000000.0}]}
